## Dynamic Flow

### Phase-I

In [6]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
from langchain_google_genai import ChatGoogleGenerativeAI

In [9]:
from dotenv import load_dotenv
import os
load_dotenv('.env')

google_api_key = os.getenv("GOOGLE_API_KEY")


In [10]:
class AppState(TypedDict):
    requirements: str
    plan: Optional[str]
    code: Optional[str]
    review: Optional[str]

In [17]:
llm= ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0)
res = llm.invoke("Write a python function that adds two numbers")

In [19]:
print(res.content)

```python
def add_numbers(num1, num2):
  """
  This function takes two numbers as input and returns their sum.

  Args:
    num1: The first number.
    num2: The second number.

  Returns:
    The sum of num1 and num2.
  """
  return num1 + num2

# Example usage:
result = add_numbers(5, 3)
print(f"The sum of 5 and 3 is: {result}")  # Output: The sum of 5 and 3 is: 8

result2 = add_numbers(10.5, 2.2)
print(f"The sum of 10.5 and 2.2 is: {result2}") # Output: The sum of 10.5 and 2.2 is: 12.7

result3 = add_numbers(-7, 15)
print(f"The sum of -7 and 15 is: {result3}") # Output: The sum of -7 and 15 is: 8
```

**Explanation:**

1.  **`def add_numbers(num1, num2):`**:
    *   `def` is the keyword used to define a function in Python.
    *   `add_numbers` is the name of the function. You can choose any valid identifier for your function name.
    *   `(num1, num2)` are the **parameters** of the function. These are placeholders for the values that will be passed into the function when it's call

### Creating Agent Phase-II

In [ ]:
from prompts import architect_prompt, orchestrator_prompt ,planner_prompt, backend_prompt, frontend_prompt, tester_prompt, reviewer_prompt

def architect_node(state: AppState):
    prompt = architect_prompt.format(requirements=state['requirements'])
    out = llm.invoke(prompt).content
    return {"plan": out}

def orchestrator_node(state: AppState):
    prompt = orchestrator_prompt.format(requirements=state['requirements'])
    out = llm.invoke(prompt).content
    return {"plan": out}

def planning_node(state: AppState):
    prompt = planner_prompt.format(plan=state['plan'])
    out = llm.invoke(prompt).content
    return {"code": out}

def backend_node(state: AppState):
    prompt = backend_prompt.format(plan=state['plan'])
    out = llm.invoke(prompt).content
    return {"code": out}

def frontend_node(state: AppState):
    prompt = frontend_prompt.format(plan=state['plan'])
    out = llm.invoke(prompt).content
    return {"code": out}

def tester_node(state: AppState):
    prompt = tester_prompt.format(code=state['code'])
    out = llm.invoke(prompt).content
    return {"review": out}


def reviewer_node(state: AppState):
    prompt = reviewer_prompt.format(code=state['code'])
    out = llm.invoke(prompt).content
    return {"review": out}


builder = StateGraph(AppState)

builder.add_node("architect", architect_node)
builder.add_node("planner", planning_node)
builder.add_node("backend", backend_node)
builder.add_node("frontend", frontend_node)
builder.add_node("tester", tester_node)
builder.add_node("reviewer", reviewer_node)

builder.add_edge("architect", "planner")
builder.add_edge("planner", "backend")
builder.add_edge("backend", "frontend")
builder.add_edge("frontend", "tester")
builder.add_edge("tester", "reviewer")
builder.add_edge("reviewer", END)

builder.set_entry_point("architect")

graph = builder.compile()
initial_state: AppState = {
    "requirements": "Create a Python function that adds two numbers.",
    "plan": None,
    "code": None,
    "review": None,
}

final_state = graph.run(initial_state)
print(final_state)